In [1]:
#coding=utf-8 
import numpy as np
import pandas as pd
import json
pd.set_option("display.precision", 2)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df = pd.read_csv('FIFA21_official_data.csv', encoding="utf-8")
col_list = df.columns.values.tolist()
for index, col in enumerate(col_list):
    print(index, " ", col)

0   ID
1   Name
2   Age
3   Photo
4   Nationality
5   Flag
6   Overall
7   Potential
8   Club
9   Club Logo
10   Value
11   Wage
12   Special
13   Preferred Foot
14   International Reputation
15   Weak Foot
16   Skill Moves
17   Work Rate
18   Body Type
19   Real Face
20   Position
21   Jersey Number
22   Joined
23   Loaned From
24   Contract Valid Until
25   Height
26   Weight
27   Crossing
28   Finishing
29   HeadingAccuracy
30   ShortPassing
31   Volleys
32   Dribbling
33   Curve
34   FKAccuracy
35   LongPassing
36   BallControl
37   Acceleration
38   SprintSpeed
39   Agility
40   Reactions
41   Balance
42   ShotPower
43   Jumping
44   Stamina
45   Strength
46   LongShots
47   Aggression
48   Interceptions
49   Positioning
50   Vision
51   Penalties
52   Composure
53   Marking
54   StandingTackle
55   SlidingTackle
56   GKDiving
57   GKHandling
58   GKKicking
59   GKPositioning
60   GKReflexes
61   Best Position
62   Best Overall Rating
63   Release Clause
64   DefensiveAwareness


In [3]:
players = json.loads(df.to_json(orient='index'))

In [7]:
player_data = []

avg_ability = {}
for j in range(27,61):
        avg_ability[ col_list[j] ] = 0;

for i in range(0, 1000):
    player = players[str(i)]
    player_ability = {}

    ability_section_name = [ 'Attack', 'Skill', 'Movement', 'Strength', 'Psychology', 'Defense', 'Goalkeeping'  ]
    for name in ability_section_name:
        player_ability[name] = {}

    for j in range(27, 61):
        if j == 56:
            if not player[ col_list[j] ]:
                player_ability['Goalkeeping'][col_list[j]] = 75.0
            else:
                player_ability['Goalkeeping'][col_list[j]] = player[ col_list[j] ]
        else:
            section_name = ability_section_name[ int((j-27) / 5) ]
            if not player[ col_list[j] ]:
                player_ability[ section_name ][ col_list[j] ] = 75.0
            else:
                player_ability[ section_name ][ col_list[j] ] = player[ col_list[j] ]
        
        avg_ability[ col_list[j] ] += player[ col_list[j] ]
    # print(json.dumps(player_ability, indent=4, ensure_ascii=False))
    player['Ability'] = player_ability
    
    player_radar_ability = {}
    player_radar_ability['Passing'] = int((player['Crossing'] + player['ShortPassing'] + player['LongPassing'])/3)
    player_radar_ability['Shooting'] = int((player['Finishing'] + player['HeadingAccuracy'] + player['Volleys'] +  player['Curve'] + player['FKAccuracy']+player['LongShots'])/6)
    player_radar_ability['Dribbling'] = int((player['Dribbling'] + player['BallControl'])/2)
    player_radar_ability['Defense'] = int((player['SlidingTackle'] + player['StandingTackle'] + player['Composure'] +  player['Positioning'] + player['Interceptions'])/5)
    player_radar_ability['Strength'] = int((player['Strength'] + player['Jumping'] + player['Stamina'] +  player['Balance'])/4)
    player_radar_ability['Speed'] = int((player['Acceleration'] + player['SprintSpeed'] + player['Agility'] +  player['Reactions'])/4)
    player['RadarAbility'] = player_radar_ability
    
    player_data.append( player )
    
for j in range(27,61):
        avg_ability[ col_list[j] ] /= 1000;
avg_radar_ability = {}
avg_radar_ability['Passing'] = int((avg_ability['Crossing'] + avg_ability['ShortPassing'] + avg_ability['LongPassing'])/3)
avg_radar_ability['Shooting'] = int((avg_ability['Finishing'] + avg_ability['HeadingAccuracy'] + avg_ability['Volleys'] +  avg_ability['Curve'] + avg_ability['FKAccuracy']+avg_ability['LongShots'])/6)
avg_radar_ability['Dribbling'] = int((avg_ability['Dribbling'] + avg_ability['BallControl'])/2)
avg_radar_ability['Defense'] = int((avg_ability['SlidingTackle'] + avg_ability['StandingTackle'] + avg_ability['Composure'] +  avg_ability['Positioning'] + avg_ability['Interceptions'])/5)
avg_radar_ability['Strength'] = int((avg_ability['Strength'] + avg_ability['Jumping'] + avg_ability['Stamina'] +  avg_ability['Balance'])/4)
avg_radar_ability['Speed'] = int((avg_ability['Acceleration'] + avg_ability['SprintSpeed'] + avg_ability['Agility'] +  avg_ability['Reactions'])/4)

print(json.dumps(avg_radar_ability, indent=4, ensure_ascii=False))

{
    "Passing": 75,
    "Shooting": 67,
    "Dribbling": 77,
    "Defense": 69,
    "Strength": 73,
    "Speed": 74
}


In [26]:
with open("player_data.json", "w") as f:
    f.write(json.dumps(player_data, ensure_ascii=False, indent=4, separators=(',', ':')))